# memstack-agent-ui 核心类型

本 notebook 演示 memstack-agent-ui 前端框架的核心类型定义。

## 概述

@memstack-agent-ui/core 包提供：
- **EventType** - 100+ 种事件类型联合
- **EventCategory** - 事件分类
- **ConversationState** - 会话状态接口
- **工具函数** - isDeltaEvent, isTerminalEvent, isHITLEvent

## 1. EventType 类型定义

EventType 是所有可能事件类型的联合类型，包含 100+ 种事件。

In [ ]:
# 在 TypeScript 中，EventType 定义如下：

event_type_definition = '''
export type EventType =
  // Core Agent Lifecycle
  | 'start'
  | 'complete'
  | 'error'
  | 'status'
  | 'cancelled'
  | 'retry'

  // Thinking
  | 'thought'
  | 'thought_delta'

  // Execution
  | 'act'
  | 'act_delta'
  | 'observe'

  // Messaging
  | 'message'
  | 'user_message'
  | 'assistant_message'
  | 'text_start'
  | 'text_delta'
  | 'text_end'

  // HITL - Permission
  | 'permission_asked'
  | 'permission_replied'

  // HITL - Clarification
  | 'clarification_asked'
  | 'clarification_answered'

  // HITL - Decision
  | 'decision_asked'
  | 'decision_answered'

  // Skills (L2 Layer)
  | 'skill_matched'
  | 'skill_execution_start'
  | 'skill_tool_start'
  | 'skill_tool_result'
  | 'skill_execution_complete'
  | 'skill_fallback'

  // Plan Mode
  | 'plan_mode_enter'
  | 'plan_mode_exit'
  | 'plan_created'
  | 'plan_updated'
  // ... 还有更多
'''

print(event_type_definition)

## 2. 事件分类

In [ ]:
# 事件按类别组织

event_categories = {
    "agent": [
        "start", "complete", "error", "status", "cancelled", "retry",
        "thought", "thought_delta",
        "act", "act_delta", "observe",
        "skill_matched", "skill_execution_start", "skill_execution_complete",
    ],
    "hitl": [
        "permission_asked", "permission_replied",
        "clarification_asked", "clarification_answered",
        "decision_asked", "decision_answered",
        "env_var_requested", "env_var_provided",
    ],
    "message": [
        "message", "user_message", "assistant_message",
        "text_start", "text_delta", "text_end",
    ],
    "sandbox": [
        "sandbox_created", "sandbox_terminated", "sandbox_status",
        "desktop_started", "desktop_stopped",
        "terminal_started", "terminal_stopped",
    ],
    "system": [
        "cost_update",
        "context_compressed", "context_status",
    ],
}

print("Event Categories:")
print("=" * 50)
for category, events in event_categories.items():
    print(f"\n{category.upper()} ({len(events)} events):")
    for event in events:
        print(f"  - {event}")

## 3. ConversationState 接口

In [ ]:
# ConversationState 定义了每个会话的完整状态

conversation_state_interface = '''
export interface ConversationState {
  // Timeline & Messages
  timeline: TimelineEvent[];
  hasEarlier: boolean;
  earliestTimeUs: number | null;
  earliestCounter: number | null;

  // Streaming State
  isStreaming: boolean;
  streamStatus: StreamStatus;
  streamingAssistantContent: string;
  error: string | null;

  // Agent Execution State
  agentState: AgentState;
  currentThought: string;
  streamingThought: string;
  isThinkingStreaming: boolean;
  activeToolCalls: Map<string, ActiveToolCall>;
  pendingToolsStack: string[];

  // Plan Mode State
  isPlanMode: boolean;

  // Agent Tasks
  tasks: AgentTask[];

  // HITL State
  pendingClarification: ClarificationAskedEventData | null;
  pendingDecision: DecisionAskedEventData | null;
  pendingEnvVarRequest: EnvVarRequestedEventData | null;
  pendingPermission: PermissionAskedEventData | null;
  doomLoopDetected: DoomLoopDetectedDataType | null;
  pendingHITLSummary: HITLSummary | null;

  // Cost Tracking
  costTracking: CostTracking | null;

  // Suggestions
  suggestions: string[];

  // MCP App Context
  appModelContext: Record<string, unknown> | null;
}
'''

print(conversation_state_interface)

## 4. 创建默认状态

In [ ]:
# 使用工厂函数创建默认状态

create_default_state = '''
import { createDefaultConversationState } from '@memstack-agent-ui/core';

// 创建默认会话状态
const state = createDefaultConversationState();

console.log(state);
// {
//   timeline: [],
//   isStreaming: false,
//   streamStatus: 'idle',
//   streamingAssistantContent: '',
//   error: null,
//   agentState: 'idle',
//   currentThought: '',
//   streamingThought: '',
//   isThinkingStreaming: false,
//   activeToolCalls: new Map(),
//   pendingToolsStack: [],
//   isPlanMode: false,
//   tasks: [],
//   pendingClarification: null,
//   pendingDecision: null,
//   pendingEnvVarRequest: null,
//   pendingPermission: null,
//   doomLoopDetected: null,
//   pendingHITLSummary: null,
//   costTracking: null,
//   suggestions: [],
//   appModelContext: null,
// }
'''

print(create_default_state)

## 5. Delta 事件检测

In [ ]:
# Delta 事件需要批处理以优化性能

delta_event_check = '''
import { isDeltaEvent } from '@memstack-agent-ui/core';

// Delta 事件类型
const events = ['text_delta', 'thought_delta', 'act_delta', 'thought', 'complete'];

events.forEach(eventType => {
  if (isDeltaEvent(eventType)) {
    console.log(`${eventType} is a delta event - should be batched`);
  } else {
    console.log(`${eventType} is not a delta event`);
  }
});

// Output:
// text_delta is a delta event - should be batched
// thought_delta is a delta event - should be batched
// act_delta is a delta event - should be batched
// thought is not a delta event
// complete is not a delta event
'''

print(delta_event_check)

## 6. 终端事件检测

In [ ]:
# 终端事件表示流结束

terminal_event_check = '''
import { isTerminalEvent } from '@memstack-agent-ui/core';

const events = ['complete', 'error', 'cancelled', 'thought', 'observe'];

events.forEach(eventType => {
  if (isTerminalEvent(eventType)) {
    console.log(`${eventType} is terminal - stream ended`);
  }
});

// Output:
// complete is terminal - stream ended
// error is terminal - stream ended
// cancelled is terminal - stream ended
'''

print(terminal_event_check)

## 7. HITL 事件检测

In [ ]:
# HITL 事件需要用户交互

hitl_event_check = '''
import { isHITLEvent } from '@memstack-agent-ui/core';

const events = [
  'permission_asked',
  'clarification_asked',
  'decision_asked',
  'env_var_requested',
  'thought',
  'observe',
];

const hitlEvents = events.filter(isHITLEvent);

console.log('HITL events requiring user interaction:');
hitlEvents.forEach(event => console.log(`  - ${event}`));

// Output:
// HITL events requiring user interaction:
//   - permission_asked
//   - clarification_asked
//   - decision_asked
//   - env_var_requested
'''

print(hitl_event_check)

## 8. 类型导入示例

In [ ]:
# 完整的类型导入

import_example = '''
// 从 @memstack-agent-ui/core 导入所有类型
import type {
  // 事件类型
  EventType,
  EventCategory,
  EVENT_CATEGORIES,

  // 会话状态
  ConversationState,
  StreamStatus,
  HITLSummary,

  // Agent 状态
  AgentState,
  ActiveToolCall,
  CostTracking,

  // 会话类型
  TimelineEvent,
  Message,
} from '@memstack-agent-ui/core';

// 导入工具函数
import {
  createDefaultConversationState,
  getHITLSummaryFromState,
  isDeltaEvent,
  isTerminalEvent,
  isHITLEvent,
  SAVE_TRIGGER_EVENTS,
  COST_UPDATE_EVENTS,
} from '@memstack-agent-ui/core';
'''

print(import_example)

## 9. 事件类别映射

In [ ]:
# EVENT_CATEGORIES 提供事件到类别的映射

event_categories_usage = '''
import { EVENT_CATEGORIES } from '@memstack-agent-ui/core';

// 获取事件的类别
const category = EVENT_CATEGORIES['permission_asked'];
console.log(category); // 'hitl'

// 按类别过滤事件
const eventTypes = ['thought', 'permission_asked', 'text_delta', 'sandbox_created'] as const;

const eventsByCategory = eventTypes.reduce((acc, type) => {
  const cat = EVENT_CATEGORIES[type];
  if (!acc[cat]) acc[cat] = [];
  acc[cat].push(type);
  return acc;
}, {} as Record<string, string[]>);

console.log(eventsByCategory);
// {
//   agent: ['thought', 'text_delta'],
//   hitl: ['permission_asked'],
//   sandbox: ['sandbox_created']
// }
'''

print(event_categories_usage)

## 总结

本 notebook 演示了 @memstack-agent-ui/core 包的核心类型：

1. **EventType** - 100+ 种事件类型联合
2. **EventCategory** - 5 种事件分类
3. **ConversationState** - 完整的会话状态接口
4. **工具函数**:
   - `isDeltaEvent()` - 检测流式增量事件
   - `isTerminalEvent()` - 检测终端事件
   - `isHITLEvent()` - 检测需要用户交互的事件
   - `createDefaultConversationState()` - 创建默认状态
   - `getHITLSummaryFromState()` - 提取 HITL 摘要

下一步：查看 `fe-02-websocket-client.ipynb` 了解 WebSocket 客户端使用。